<a href="https://colab.research.google.com/github/Patrickslide/Geospatial-analysis-project/blob/main/1_Distance_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [2]:
pip install PyQt5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.4/361.4 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 8.8 MB/s eta 0:00:00


In [3]:
import warnings

In [4]:
# # try:
#   import mapclassify
# except ModuleNotFoundError as e:
#   !pip install mapclassify
#   import mapclassify

In [5]:
try:
  import pygeos
except ModuleNotFoundError as e:
  !pip install pygeos
  import pygeos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.6 MB/s eta 0:00:00


In [ ]:
try:
  import pyrosm
except ModuleNotFoundError as e:
  !pip install pyrosm
  import pyrosm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.8 MB/s eta 0:00:00
  Using cached pyrobuf-0.9.3-cp38-cp38-linux_x86_64.whl
  Using cached cykhash-2.0.0-cp38-cp38-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.2 MB/s eta 0:00:00


In [ ]:
# try:
#     import mapclassify
# except ModuleNotFoundError as e:
#     !pip install mapclassify
#     import mapclassify


In [ ]:
try:
  import numpy
except ModuleNotFoundError as e:
  !pip install numpy
  import numpy

In [ ]:
try:
  import shapely
except ModuleNotFoundError as e:
  !pip install shapely
  import shapely

In [ ]:
try:
  import osmnx as ox
except ModuleNotFoundError as e:
  !pip install osmnx
  import osmnx as ox

In [ ]:
try:
  import geopandas
except ModuleNotFoundError as e:
  !pip install geopandas
  import geopandas

In [ ]:
try:
  import python-igraph
except ModuleNotFoundError as e:
  !pip install python-igraph
  import python-igraph

In [ ]:
import requests
import geopandas as gpd
import fiona
import urllib
import os
import zipfile, io
import urllib3
import ssl
import shutil
import matplotlib
from matplotlib import pyplot as plt
import networkx as nx


Now for the functions required to interact with Istat using colab:

In [ ]:

class CustomHttpAdapter(requests.adapters.HTTPAdapter):
  def __init__(self, ssl_context=None, **kwargs):
    self.ssl_context = ssl_context
    super().__init__(**kwargs)

  def init_poolmanager(self, connections, maxsize, block=False):
    self.poolmanager = urllib3.poolmanager.PoolManager(num_pools=connections, maxsize=maxsize, block=block, ssl_context=self.ssl_context)

def get_legacy_session():
  ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
  ctx.options |= 0x4  
  session = requests.session()
  session.mount('https://', CustomHttpAdapter(ctx))
  return session

def download_file(url):
# source: https://stackoverflow.com/a/33488338
  local_filename = url.split('/')[-1]
  with get_legacy_session() as s:
    r = s.get(url, stream=True)
    r.raw.decode_content = True
    with open(local_filename, 'wb') as f:
      shutil.copyfileobj(r.raw, f)
  return local_filename

In [ ]:
if not os.path.exists('Limiti01012022'):
    zip_file_url = 'https://www.istat.it/storage/cartografia/confini_amministrativi/non_generalizzati/Limiti01012022.zip'
    zip_file_name = "Limiti01012022.zip"
    
    try: 
      urllib.request.urlretrieve(zip_file_url ,zip_file_name) 
    
    except:
      zip_file_name = download_file(zip_file_url)

    z = zipfile.ZipFile(zip_file_name)
    # unzip the file
    z.extractall()

In [ ]:
os.listdir('Limiti01012022')
os.chdir('Limiti01012022')
os.chdir('Com01012022')

for root, dirs, files in os.walk("."):  
    for filename in files:
        print(filename)

In [ ]:
cities=gpd.read_file('Com01012022_WGS84.shp')

## Start

I want to compute the distance between two cities, assuming one travel by car.
First, we set as input one location and we localize it, plotting it and finding the province it belongs to. I'm using simplemaps's list of italian cities, available here:
https://simplemaps.com/data/it-cities

For now I try only cities belonging to the same region, as I tried using the whole country but it would require many Gb of RAM and storing capacities.

In [ ]:
start = input("Starting address: ")
point_1 = gpd.tools.geocode(start, provider="arcgis")    

Here, make sure that we are selecting an italian city!!

Nominatim is an API which allows us to know the province, region and country of any given location.

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.reverse(str(float(point_1.geometry.y))+","+ str(float(point_1.geometry.x)))

start_address = location.raw['address']
#We get the province.

if start_address['country'] != 'Italia':
  print('Restart! Not an italian city')
print('Provincia:', start_address['county'])

In [ ]:
start_address

In [ ]:
location

In [ ]:
city_start = cities[cities['COMUNE'] == start_address['city']]
city_start.plot()
print('Starting City: ', start_address['city'])
plt.show()


Now, I take a local map of car roads in order to be able to find the shortest route between two cities; the next step will be to compute its cost.

In [ ]:
# ox.config(log_console=True)
# gdf = ox.geocode_to_gdf('Italy')
# gdf.plot()

In [ ]:
area = start_address['county']
#USE COUNTY FOR TRIAL RUNS, STATE FOR THE ACTUAL DEMO.
area

In [ ]:
G = ox.graph_from_place(area, network_type='drive') #Only need to be ran once.

In [ ]:
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
ox.save_graphml(G, "region_roads.graphml")

In [ ]:
G = ox.load_graphml("region_roads.graphml")

In [ ]:
point_nearest_start = ox.distance.nearest_nodes(G,Y=location[1][0], X=location[1][1])

## End

In [ ]:
start = input("Destination: ")
point_2 = gpd.tools.geocode(start, provider="arcgis")

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")
location2 = geolocator.reverse(str(float(point_2.geometry.y))+","+ str(float(point_2.geometry.x)))

end_address = location2.raw['address']

In [ ]:
location2.raw

In [ ]:
if start_address['county'] !=  end_address['county']:
  print('Not from the same province!')

In [ ]:
city_end = cities[cities['COMUNE'] == end_address['town']]
city_end.plot()
print('Arriving city: ', end_address['town'])
plt.show()

In [ ]:
# province_end = provinces[provinces['DEN_PROV'] == end_address['county']]
# province_end

In [ ]:
point_nearest_end = ox.distance.nearest_nodes(G,Y=location[1][0], X=location[1][1])

## Travel

In [ ]:
distance = point_1.to_crs('epsg:32632').geometry.distance(point_2.geometry.to_crs('epsg:32632')).values[0] / 1000
distance = round(distance, 3)
print('Distance: ', distance, 'km')

Here I plot the shortest route by distance.

In [ ]:
shortest_path_distance = ox.shortest_path(G, point_nearest_start, point_nearest_end, weight='length')
travel_time = nx.shortest_path_length(G, point_nearest_start, point_nearest_end, weight='travel_time')
minutes = (travel_time * 60) // 100 
seconds = round((travel_time * 60) % 100*0.6)

km_distance = round(nx.shortest_path_length(G, point_nearest_start, point_nearest_end, weight='length')/1000, 3)

In [ ]:
print('Expected Travel time: ', minutes, 'minutes and ', seconds, 'seconds')      #nx distance
print('Distance: ', km_distance,'km')
fig, ax = ox.plot_graph_route(G, shortest_path_distance, route_color='green', route_linewidth=6, node_size=0)

Now, I want to compute the travel cost; assuming one goes by car, I searched online for answers and for this project I'm assuming an average consumption of 1 liter of fuel for each 12.5 km travelled and a cost of fuel of 1.65 $ per liter.

Highway assumes an average speed of 100 km/h, freeway 70 km/h.

In [ ]:
road_type = input('highway or freeway?')

if road_type.lower() == 'highway':
  minute_time = km_distance*60//100
  seconds_time = round((travel_time * 60) % 100*0.6)
  time= str(minute_time) + 'minutes, ' + str(seconds_time) + 'seconds'

if road_type.lower() == 'freeway':
  minute_time = km_distance*60//70
  seconds_time = round((travel_time * 60) % 100*0.6)
  time= str(minute_time) + 'minutes, ' + str(seconds_time) + 'seconds'

In [ ]:
price = 1.65
cost = 1.65*km_distance
print('Car travel information: time =', time, 'price = ', price, '€')